In [1]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
train_data_dir = "/Users/deepanshudubb/Documents/projects/accident/training"
val_data_dir = "/Users/deepanshudubb/Documents/projects/accident/validation"

In [3]:
img_width, img_height = 150, 150

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary', 
    shuffle=False 
)

Found 1840 images belonging to 2 classes.


In [6]:
validation_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 460 images belonging to 2 classes.


In [7]:
def extract_features(generator):
    features = []
    labels = []
    for _ in range(generator.samples // generator.batch_size):
        batch_x, batch_y = next(generator)
        features.append(batch_x.reshape(batch_x.shape[0], -1))
        labels.append(batch_y)
    return np.concatenate(features), np.concatenate(labels)

In [8]:
train_features_knn, train_labels_knn = extract_features(train_generator)
val_features_knn, val_labels_knn = extract_features(validation_generator)


In [9]:

knn = KNeighborsClassifier(n_neighbors=5) 
knn.fit(train_features_knn, train_labels_knn)

KNeighborsClassifier()

In [10]:
knn_predictions = knn.predict(val_features_knn)
knn_accuracy = accuracy_score(val_labels_knn, knn_predictions)
print("KNN Accuracy:", knn_accuracy)

KNN Accuracy: 0.6183035714285714


In [11]:
model_cnn = Sequential()

In [12]:
optimizer = Adam(learning_rate=0.0001)
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [13]:

model_cnn.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model_cnn.add(BatchNormalization()) 
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Conv2D(128, (3, 3), activation='relu'))
model_cnn.add(BatchNormalization())
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

model_cnn.add(Flatten())
model_cnn.add(Dense(256, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(1, activation='sigmoid'))


optimizer = Adam(learning_rate=0.0001) 
model_cnn.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

epochs = 50

model_cnn.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)

/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 352ms/step - accuracy: 0.5279 - loss: 3.3965 - val_accuracy: 0.4866 - val_loss: 1.0352
Epoch 2/50
 1/57 ━━━━━━━━━━━━━━━━━━━━ 18s 329ms/step - accuracy: 0.7812 - loss: 0.7340

/Users/deepanshudubb/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7812 - loss: 0.7340 - val_accuracy: 0.4866 - val_loss: 1.0250
Epoch 3/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 347ms/step - accuracy: 0.5814 - loss: 0.6817 - val_accuracy: 0.4866 - val_loss: 1.0399
Epoch 4/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5312 - loss: 0.7385 - val_accuracy: 0.4866 - val_loss: 1.0262
Epoch 5/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 349ms/step - accuracy: 0.6179 - loss: 0.6505 - val_accuracy: 0.6161 - val_loss: 0.7236
Epoch 6/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9062 - loss: 0.5452 - val_accuracy: 0.6094 - val_loss: 0.7373
Epoch 7/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 354ms/step - accuracy: 0.6325 - loss: 0.6559 - val_accuracy: 0.6451 - val_loss: 0.7207
Epoch 8/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7500 - loss: 0.3894 - val_accuracy: 0.6451 - val_loss: 0.7257
Epoch 9/50
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 356ms/step - accuracy: 0.6550 - loss: 0.6230 - val_accuracy: 0.6004 - val_

In [14]:
loss, accuracy = model_cnn.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)

print("CNN Accuracy:", accuracy)

14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 0.7955 - loss: 0.5623
CNN Accuracy: 0.8058035969734192


In [15]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array 
import numpy as np

In [16]:

def predict_accident(image_path, model):
    img = load_img(image_path, target_size=(img_width, img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255. 

    prediction = model.predict(img)
    if prediction[0][0] > 0.5:
        return "Noxxn-Accidental"
    else:
        return "Accidental"

In [18]:
test_image_path = "/Users/deepanshudubb/Documents/projects/accident/fine.jpg"
result_cnn = predict_accident(test_image_path, model_cnn)
print("CNN Prediction:", result_cnn)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
CNN Prediction: Noxxn-Accidental


In [19]:
model_cnn.save("cnn_model.h5")

In [20]:
import pickle
with open("knn_model.pkl", "wb") as f:
    pickle.dump(knn, f) 